In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [24]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [25]:
n_vocab=len(set(notes))

In [26]:
sequence_length = 500
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [27]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=10, verbose=1, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
15185/15185 [==============================] - ETA: 4511s - loss: 5.40 - ETA: 4493s - loss: 5.40 - ETA: 4646s - loss: 5.37 - ETA: 4850s - loss: 5.52 - ETA: 5002s - loss: 5.47 - ETA: 5180s - loss: 5.40 - ETA: 5334s - loss: 5.36 - ETA: 5523s - loss: 5.29 - ETA: 5742s - loss: 5.24 - ETA: 5963s - loss: 5.23 - ETA: 6176s - loss: 5.21 - ETA: 6357s - loss: 5.19 - ETA: 6559s - loss: 5.17 - ETA: 6758s - loss: 5.15 - ETA: 6936s - loss: 5.13 - ETA: 7110s - loss: 5.10 - ETA: 7236s - loss: 5.08 - ETA: 7357s - loss: 5.06 - ETA: 7450s - loss: 5.05 - ETA: 7560s - loss: 5.05 - ETA: 7652s - loss: 5.05 - ETA: 7720s - loss: 5.04 - ETA: 7832s - loss: 5.04 - ETA: 7914s - loss: 5.03 - ETA: 7985s - loss: 5.02 - ETA: 8043s - loss: 5.02 - ETA: 8095s - loss: 5.01 - ETA: 8136s - loss: 5.01 - ETA: 8194s - loss: 5.00 - ETA: 8245s - loss: 5.00 - ETA: 8304s - loss: 4.99 - ETA: 8324s - loss: 4.97 - ETA: 8346s - loss: 4.97 - ETA: 8375s - loss: 4.96 - ETA: 8406s - loss: 4.95 - ETA: 8409s - loss: 4.94 - ETA: 8

In [15]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [16]:
# Load the weights to each node
model.load_weights('weights-improvement-09-3.6366-bigger.hdf5')

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(50):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [21]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [23]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='Music_5MidiFiles.mid')

'Music_5MidiFiles.mid'